In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import pandas as pd
import requests
import numpy as np
import os
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from typing import Tuple, Optional

def process_image(image_path: str, resize: bool = True) -> Tuple[tf.Tensor, Optional[str]]:
    if image_path.startswith('http'):
        response = requests.get(image_path)
        if response.status_code == 200:
            img_data = BytesIO(response.content)
            img = cv2.imdecode(np.frombuffer(img_data.read(), np.uint8), -1)
            if img is not None:
                if resize:
                    img = cv2.resize(img, (300, 300))
                return tf.convert_to_tensor(img), image_path
    elif os.path.exists(image_path):
        img = cv2.imread(image_path)
        if img is not None:
            if resize:
                img = cv2.resize(img, (300, 300))
            return tf.convert_to_tensor(img), image_path
    return None, None

model_url = "https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1"
model = hub.load(model_url)

caminho_avaliacao = r"D:\imagesapp\images\avaliacao.csv"
avaliacao = pd.read_csv(caminho_avaliacao, sep=";", encoding="utf-8")

caminho_objetos = r"D:\imagesapp\images\objetos.csv"
objetos_df = pd.read_csv(caminho_objetos, sep=";", encoding="utf-8")

output_dir = r"D:\imagesapp\images\output"
os.makedirs(output_dir, exist_ok=True)

for index, row in avaliacao.iterrows():
    image_url = row['imagem']

    img, image_path = process_image(image_url)
    
    if img is not None:
        img = img[tf.newaxis, ...]
        detections = model(img)

        boxes = detections["detection_boxes"][0].numpy()
        scores = detections["detection_scores"][0].numpy()
        classes = detections["detection_classes"][0].numpy().astype(int)

        fig, ax = plt.subplots(1)
        ax.imshow(cv2.cvtColor(img[0].numpy(), cv2.COLOR_BGR2RGB))

        objetos_detectados = [] 

        for i in range(len(boxes)):
            if scores[i] > 0.5:
                classe_detectada = classes[i]
                objetos_correspondentes = objetos_df.loc[objetos_df['objetos'] == classe_detectada, 'objetos'].values
                if len(objetos_correspondentes) > 0:
                    objeto_detectado = objetos_correspondentes[0]
                else:
                    objeto_detectado = f"Classe {classe_detectada}"
                ymin, xmin, ymax, xmax = boxes[i]
                height, width = img.shape[1], img.shape[2]
                ymin, xmin, ymax, xmax = int(ymin * height), int(xmin * width), int(ymax * height), int(xmax * width)
                
                rect = Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red')
                ax.add_patch(rect)
                objetos_detectados.append(objeto_detectado)

        plt.axis('off')
        objetos_detectados_text = ', '.join(objetos_detectados)
        if len(objetos_detectados_text) > 50: 
            objetos_detectados_text = '\n'.join(objetos_detectados_text.split(', '))
        plt.text(10, 10, f"Objetos Detectados:\n{objetos_detectados_text}", color='red', fontsize=8, bbox=dict(fill=False, color='white'))
        plt.show()

print("Imagens com bounding boxes exibidas.")



In [ ]:
avaliacao = pd.read_csv(caminho_avaliacao, sep=";", encoding="utf-8")
avaliacao.columns

['nome', 'imagem']

Index(['nome', 'imagem'], dtype='object')

In [ ]:
avaliacao = pd.read_csv(caminho_avaliacao, sep=";", encoding="utf-8")

avaliacao.columns

Index(['nome', 'imagem'], dtype='object')

In [ ]:
objetos_df = pd.read_csv(r"D:\imagesapp\images\objetos.csv", sep=";", encoding="utf-8")
objetos_df.columns
(['objetos', 'urlimagem', 'referencia'],

Index(['objetos', 'urlimagem', 'referencia'], dtype='object')